In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import datetime
import time
import os
import pathlib
import matplotlib.pyplot as plt
 
# Some stuff to make utils-function work
import sys
sys.path.append('../utils')
from pipeline import create_dataset, split_and_create_dataset, prepare_for_training
from utils import show_image, class_distribution
from utils import print_split_info, unpipe, tf_bincount, checkout_dataset
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

project_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
data_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/labeled_ttv/')
unlab_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/unlabeled_ttv/')

iteration = 0
model_name = str(iteration)+"_teacher"
log_dir = "./logs/{}/{}".format(project_time, model_name)

conf = {
    # Dataset
    "data_dir": data_dir,
    "unlab_dir": unlab_dir,
    "log_dir": log_dir,
    "cache_dir": "./cache",
    "ds_info": 'hypkva',
    "augment": ["xcrop","flip","brightness","saturation","contrast","rotate"],
    "aug_mult": 0.7,
    "resample": False,
    "class_weight": False,
    "shuffle_buffer_size": 2000,        # 0=no shuffling
    "seed": 2511,
    "neg_class": None,                 # select neg class for binary ds (normal class)
    "outcast": None,                   # list of folders to drop - currently only works for 1 item
    # Model
    "model_name": model_name,
    "model": 'EfficientNetB4',
    "weights": None,                   # which weights to initialize the model with
    "dropout": 0.2,
    "num_epochs": 4,
    "batch_size": 16,
    "img_shape": (128, 128, 3),
    "learning_rate": 0.001,
    "optimizer": 'Adam',
    "final_activation": 'softmax',     # sigmoid for binary ds
    # Callbacks
    "learning_schedule": False,
    "decay_rate": 0,                   # 128:0.25   64:1.0   32:4.0   16:16   8:64
    "checkpoint": False,
    "early_stopp": False,
    "early_stopp_patience": 7,
    # Misc
    "verbosity": 1,
    "keep_threshold": 0.0
    }

In [ ]:
ds = create_dataset(conf)

In [ ]:
import pickle
from pipeline import create_unlab_ds

from utils import print_bar_chart
from utils import checkout_findings
from utils import checkout_class

from unlabeled_utils import custom_sort
from unlabeled_utils import resample_unlab
from unlabeled_utils import generate_labels
from unlabeled_utils import reduce_dataset

pickle_path = "/home/henriklg/master-thesis/code/hyper-kvasir/experiments/unlab_dist"
unlab_findings = pickle.load (open (pickle_path+"/unlab_findings.pkl", "rb"))

In [ ]:
# Sort in order of highest confidence to lowest
unlab_findings = custom_sort(unlab_findings)

checkout_findings(unlab_findings, conf)

In [ ]:
# Print grid of images from one of the classes
checkout_class("ulcerative-colitis-grade-3", unlab_findings, conf)